In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import bert
from bert import tokenization
from bert import run_classifier
import json
from pandas import HDFStore, read_hdf
import pandas as pd
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)



TF version:  1.15.0
Hub version:  0.8.0


In [2]:
DATA_COLUMN = 'ServiceDescription'
LABEL_COLUMN = 'ServiceClassification'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [3]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [4]:
dft = pd.read_csv('train.csv')
trainData = dft.to_json(orient='records')
trainData = json.loads(trainData)
finalTraindata = pd.DataFrame(trainData)
finalTraindata= finalTraindata.sample(frac=1)
h5f = HDFStore('train.h5')
h5f.put('Train', finalTraindata, format='table')
h5f.close()
finalTraindata = read_hdf('train.h5', key='Train')
finalTraindata['ServiceDescriptionLength'] = finalTraindata['ServiceDescription'].str.len()
finalTraindata.sort_values('ServiceDescriptionLength', ascending=True, inplace=True)
finalTraindata

,ServiceName,ServiceDescription,ServiceClassification,ServiceDescriptionLength
7216,Mapy API,Czech mapping API.,Mapping,18
388,Zatune API,This API has retired.,Music,21
692,Webcams.travel API,The World's Largest Webcam API,Travel,30
1874,bLaugh Comics API,MAD Magazine for the Blogosphere.,Other,33
1676,Yandex API,XML API for Russian search engine.,Search,34
...,...,...,...,...
6523,Nokia Maps API,What Is the Maps API? The Nokia Maps API for J...,Mapping,1665
78,Hellenic Bank Account Reporting API,The Hellenic Bank Account Reporting API will h...,Banking,1715
7090,Talis API,The Talis Platform is open to any software dev...,Database,1891
8011,Samepoint Real-Time Social Media API,Samepoint has released Version 2 of its Social...,Social,2131


In [5]:
df = pd.read_csv('test.csv')

# 将 DataFrame 转换为 JSON 字符串
data = df.to_json(orient='records')


# 解析 JSON 字符串为 JSON 对象（Python 字典或列表）
data = json.loads(data)

finaldata = pd.DataFrame(data)

finaldata= finaldata.sample(frac=1)

finaldata

,ServiceName,ServiceDescription,ServiceClassification
224,Kobiton API,The Kobiton mobile cloud platform supports the...,Cloud
1093,Redbox Widgets API,Redbox provides automated movie and video game...,Entertainment
1504,CO-OPS API,"As part of NOAA, CO-OPS (Center for Operationa...",Science
1198,MyMemory HTS API,MyMemory is the world's largest Translation Me...,Tools
1187,ExactTarget Fuel REST API,The ExactTarget Fuel REST API provides develop...,Cloud
...,...,...,...
2077,Yendo Accounts API,The Yendo Accounts developer API allows regist...,Financial
342,SpinnerChief API,SpinnerChief is designed to generate unique ar...,Tools
2056,entreprise-facile API,REST-type API that allows you to create automa...,Enterprise
1850,weship(io) API,weship(io) is a shipment tracking service and ...,Shipping


In [6]:

h5f = HDFStore('test.h5')
h5f.put('Test', finaldata, format='table')
h5f.close()
finaldata = read_hdf('test.h5', key='Test')
print(finaldata.shape)
finaldata['ServiceDescriptionLength'] = finaldata['ServiceDescription'].str.len()
finaldata.sort_values('ServiceDescriptionLength', ascending=True, inplace=True)
finaldata

(2210, 3)


,ServiceName,ServiceDescription,ServiceClassification,ServiceDescriptionLength
1064,Open Patent Services API,"Patent, family, legal, bibliographic data, and...",Government,83
1385,Pixenate API,Pixenate is the photo editor of choice for pho...,Photos,87
1510,Yahoo Related Suggestions API,The Related Suggestion service provides sugges...,Search,95
1128,Cafe Press API,Allows developers to make queries and run sear...,eCommerce,98
600,PhishTank API,PhishTank is a free community site where anyon...,Security,98
...,...,...,...,...
926,HUSAR Bioinformatics Lab API,The HUSAR Bioinformatics Lab at DKFZ provides ...,Science,1482
294,Hotwire API,Hotwire is a discount travel site. Hotwire hel...,Travel,1498
1022,GeoRanker API,GeoRanker enables users to view search engines...,Search,1566
1588,iGeolise Travel Time API,The TravelTime platform API makes location-bas...,Transportation,1747


In [9]:
label_list=list(set(finalTraindata['ServiceClassification']))
label_list[39]

'Transportation'

In [ ]:



def prepareData_BERT_MiniBatch(test, batchsize = 32, maxDesLength = 110, maxNameLength = 10):

  testdata = []

  for i, g in test.groupby(np.arange(len(test)) // batchsize):
      # print (g)

      test1_InputExamples = g.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                        text_a = x['ServiceDescription'], 
                                                                        text_b = None, 
                                                                        label = x[LABEL_COLUMN]), axis = 1)

      maxLengthInMiniBatch = len(test1_InputExamples.iloc[-1].text_a)
      print("MiniBatch MaxLength: " + str(maxLengthInMiniBatch))
      
      test2_InputExamples = g.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                        text_a = x['ServiceName'], 
                                                                        text_b = None, 
                                                                        label = x[LABEL_COLUMN]), axis = 1)
      

      #     # We'll set sequences to be at most 128 tokens long.
     
      # # Convert our train and test features to InputFeatures that BERT understands.
      if maxLengthInMiniBatch >= maxDesLength:
         test1_features = bert.run_classifier.convert_examples_to_features(test1_InputExamples, label_list, maxDesLength, tokenizer)
      else:
         test1_features = bert.run_classifier.convert_examples_to_features(test1_InputExamples, label_list, maxLengthInMiniBatch, tokenizer)

      test2_features = bert.run_classifier.convert_examples_to_features(test2_InputExamples, label_list, maxNameLength, tokenizer)    

      X1_test = np.array([o.input_ids for o in test1_features])
      X1_mask_test = np.array([o.input_mask for o in test1_features])
      X1_segment_test = np.array([o.segment_ids for o in test1_features])

      X2_test = np.array([o.input_ids for o in test2_features])
      X2_mask_test = np.array([o.input_mask for o in test2_features])
      X2_segment_test = np.array([o.segment_ids for o in test2_features])

      Y_test = np.array([o.label_id for o in test2_features])
      print(Y_test[0])
      Test_Y_one_hot = convert_to_one_hot(Y_test, 50)
      print(Test_Y_one_hot[0])

      data = [X1_test, X1_mask_test, X1_segment_test, X2_test, X2_mask_test, X2_segment_test], Test_Y_one_hot
    
      testdata.append(data)
  
  return testdata
testdata = prepareData_BERT_MiniBatch(finaldata, batchsize = 32) 

In [ ]:
import pickle
  
f = open('test.pickle', 'wb')
pickle.dump(testdata, f)
f.close()

In [ ]:


validateData=[]
with open('./validate.json', 'r') as validate_file:
    validateData = json.load(validate_file)
print(validateData[0])
label_list = set(validateData)

In [ ]:



def prepareData_BERT_MiniBatch(test, batchsize = 32, maxDesLength = 110, maxNameLength = 10):

  testdata = []

  for i, g in test.groupby(np.arange(len(test)) // batchsize):
      # print (g)

      test1_InputExamples = g.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                        text_a = x['ServiceDescription'], 
                                                                        text_b = None, 
                                                                        label = x[LABEL_COLUMN]), axis = 1)

      maxLengthInMiniBatch = len(test1_InputExamples.iloc[-1].text_a)
      print("MiniBatch MaxLength: " + str(maxLengthInMiniBatch))
      
      test2_InputExamples = g.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                        text_a = x['ServiceName'], 
                                                                        text_b = None, 
                                                                        label = x[LABEL_COLUMN]), axis = 1)
      

      #     # We'll set sequences to be at most 128 tokens long.
     
      # # Convert our train and test features to InputFeatures that BERT understands.
      if maxLengthInMiniBatch >= maxDesLength:
         test1_features = bert.run_classifier.convert_examples_to_features(test1_InputExamples, label_list, maxDesLength, tokenizer)
      else:
         test1_features = bert.run_classifier.convert_examples_to_features(test1_InputExamples, label_list, maxLengthInMiniBatch, tokenizer)

      test2_features = bert.run_classifier.convert_examples_to_features(test2_InputExamples, label_list, maxNameLength, tokenizer)    

      X1_test = np.array([o.input_ids for o in test1_features])
      X1_mask_test = np.array([o.input_mask for o in test1_features])
      X1_segment_test = np.array([o.segment_ids for o in test1_features])

      X2_test = np.array([o.input_ids for o in test2_features])
      X2_mask_test = np.array([o.input_mask for o in test2_features])
      X2_segment_test = np.array([o.segment_ids for o in test2_features])

      Y_test = np.array([o.label_id for o in test2_features])
      print(Y_test[0])
      Test_Y_one_hot = convert_to_one_hot(Y_test, 50)
      print(Test_Y_one_hot[0])

      data = [X1_test, X1_mask_test, X1_segment_test, X2_test, X2_mask_test, X2_segment_test], Test_Y_one_hot
    
      testdata.append(data)
  
  return testdata